<a href="https://colab.research.google.com/github/adgregory/clibo-cloud-functions/blob/master/KuMatch/KuMatch_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
!pip install stanza

     |████████████████████████████████| 432 kB 15.7 MB/s 
     |████████████████████████████████| 170 kB 40.5 MB/s 
  Created wheel for emoji: filename=emoji-1.6.1-py3-none-any.whl size=169314 sha256=5afd7f26dd64576571cf1b05c2a8d50d77c0c4be4532bc7d909fb6213adc1974
  Stored in directory: /root/.cache/pip/wheels/ea/5f/d3/03d313ddb3c2a1a427bb4690f1621eea60fe6f2a30cc95940f
Successfully built emoji


In [ ]:
import stanza
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger') 

from nltk.stem import WordNetLemmatizer
from nltk import FreqDist
from nltk.collocations import BigramAssocMeasures, BigramCollocationFinder
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
stanza.download("es")

2021-11-03 21:54:30 INFO: Downloading default packages for language: es (Spanish)...


2021-11-03 21:54:48 INFO: Finished downloading models and saved to /root/stanza_resources.


## GSheet Upload

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

user_likes_ws = gc.open('user_likes').sheet1
descriptions_ws = gc.open('descriptions').sheet1

# get_all_values gives a list of rows.
user_likes_rows = user_likes_ws.get_all_values()
descriptions_rows = descriptions_ws.get_all_values()

# Convert to a DataFrame and render.

user_likes_df = pd.DataFrame.from_records(user_likes_rows)
descriptions_df = pd.DataFrame.from_records(descriptions_rows)

## Transformations

In [ ]:
ul_header = user_likes_df.iloc[0]
user_likes_df = user_likes_df[1:]
user_likes_df.columns = ul_header

d_header = descriptions_df.iloc[0]
descriptions_df = descriptions_df[1:]
descriptions_df.columns = d_header

In [ ]:
descriptions_df.head(5)

,rest_name,category,size,dish_name,price,dish_description
1,cafe del mar,fuertes,,ceviche cafe del mar,,"Pulpo, camaron, tilapia, agucate, cebolla, aji..."
2,cafe del mar,fuertes,,ceviche tropical,,"Camarones, cebolla, limon criollo, limon tahit..."
3,cafe del mar,fuertes,,ceviche azteca,,"Pulpo, camaron, tilapia, agucate, cebolla, aji..."
4,cafe del mar,fuertes,,ceviche tradicional,,"Tilapia, cebolla, ajinomoto, aguacate, limón c..."
5,cafe del mar,fuertes,,causa de camarones,,"Camarones, púre, aguacate, ajinomoto, cbebolla..."


In [ ]:
user_likes_df.head()

,user,rest_name,category,,dish_name,price,dish_description
1,Alvin Gregory,cafe del mar,,,pulpo a la parrila,,"Pulpo, perejil, ajo, reduccion balsamica"
2,Alvin Gregory,cafe del mar,,,ceviche tropical,,"Camarones, cebolla, limon criollo, limon tahit..."
3,Alvin Gregory,cafe del mar,,,cuasa de pulpo,,"Pulpo, púre, aguacate, ajinomoto, cebolla, mar..."
4,Alvin Gregory,Distrito Burger Bar,HAMBURGUESAS,,Distrito,19000,"Carne de Res(150g), queso holandés fundidos co..."
5,Alvin Gregory,Distrito Burger Bar,HAMBURGUESAS,,Imperial,29000,"Triple Carne de Res(450g),queso cheddar,tocine..."


## Main Functions

In [ ]:
def get_words_freq(user_description_df, max_most_common, name):
  total_vocab = set()
  for bio in user_description_df['description']:
    try:
      total_vocab.update(bio)
    except:
      total_vocab.update(str(bio))
  # Most Frequent words
  words = []
  for bio in user_description_df['description']:
    try:
      words.extend(bio)
    except:
      words.extend(str(bio))
  # Frequency of words
  bio_freq = FreqDist(words)
  # Delete words
  del bio_freq['casa'], bio_freq['incluye'], bio_freq['salsa'], bio_freq['acompañado'], bio_freq['gramo'] 
  # plt.style.use('ggplot')
  # plt.figure(figsize=(15,5))

  # plt.bar(*zip(*bio_freq.most_common(max_most_common)))
  # plt.xticks(rotation=75)
  # plt.title(name)
  # plt.show()
  return words, bio_freq

In [ ]:
def func_user_likes(name, user_likes_df, max_most_common):
  new_descriptions = []
  # NLP
  stops = stopwords.words('spanish')
  # DF Subset
  subset_df_likes = user_likes_df.loc[user_likes_df['user'] == name]
  for description in np.array(subset_df_likes['dish_description']):
    lemmatizer = nlp(description)
    lemmas = []
    for i in range(len(lemmatizer.sentences[0].words)):
      lemma = lemmatizer.sentences[0].words[i].lemma
      lemma = lemma.lower()
      xpos  = lemmatizer.sentences[0].words[i].upos
      if xpos != "PUNCT" and xpos != 'ADP' and lemma not in stops and xpos != 'ADJ' and xpos !='NUM':
        lemmas.append(lemma)
    new_descriptions.append(lemmas)
  user_description = pd.DataFrame(np.array(new_descriptions), columns = ['description'])
  words, mf_words = get_words_freq(user_description, max_most_common, name)
  mf_words = mf_words.most_common(max_most_common)
  return mf_words

In [ ]:
def get_dish_lemma(df_descriptions, restaurant, idx):
  subset_rest_descriptions = df_descriptions.loc[df_descriptions['rest_name'] == restaurant]
  rest_descriptions = np.array(subset_rest_descriptions['dish_description'])
  dish_description = rest_descriptions[idx]
  stops = stopwords.words('spanish')
  lemmatizer = nlp(dish_description)
  lemmas = []
  for i in range(len(lemmatizer.sentences[0].words)):
    lemma = lemmatizer.sentences[0].words[i].lemma
    lemma = lemma.lower()
    xpos  = lemmatizer.sentences[0].words[i].upos
    if xpos != "PUNCT" and xpos != 'ADP' and lemma not in stops and xpos != 'ADJ' and xpos !='NUM':
      lemmas.append(lemma)
  return lemmas

In [ ]:
def get_dish_match(restaurant, descriptions_df, user_dict, idx):
  user_list = list(user_dict)
  print(user_dict)
  matches = list(set(user_dict).intersection(get_dish_lemma(descriptions_df, restaurant, idx)))
  match_per = 0
  for match in matches:
    match_per += user_dict[match]
  return match_per

In [ ]:
def get_user_matches(restaurant, descriptions_df, user_dict):
  restaurant_dishes = descriptions_df.loc[descriptions_df['rest_name'] == restaurant]
  restaurant_dishes = restaurant_dishes.reset_index(drop=True)
  rest_dish_descriptions = np.array(restaurant_dishes['dish_description'])
  match_values = []
  for idx in range(len(rest_dish_descriptions)):
    match_val = get_dish_match(restaurant, descriptions_df, user_dict, idx)
    match_values.append(match_val)
  match_values = np.array(match_values)
  return match_values

In [ ]:
def replacements(sentence):
  chrs = [',', '-', '.', '/', '(']
  # Lower Case
  sentence = sentence.lower()
  # Replace Special Characters-Punctuation
  for chr in chrs:
    sentence = sentence.replace(chr, ' ')
  return sentence


def lemmatizeSentence(sentence, toExclude, toInclude):
  # Replace Punctuation
  description = replacements(sentence)
  # Lemmatizer
  nlp = stanza.Pipeline(lang='es', processors= 'tokenize, mwt, pos, lemma', verbose = False)
  lemmitizer = nlp(description)
  # Stopwords
  stops = stopwords.words('spanish')

  wordsLemma = lemmitizer.sentences[0].words
  
  if wordsLemma[0].lemma == 'car' and wordsLemma[1].lemma == 'ne':
    class Word:
      def __init__(self, id, lemma, upos):
        self.id = id
        self.lemma = lemma
        self.upos = upos
    insert = Word(0, "carne", "NOUN")
    wordsLemma = wordsLemma[2:]
    wordsLemma.insert(0, insert)

  lemma = []
  for wordLemma in wordsLemma:
    upos = wordLemma.upos
    word = wordLemma.lemma
    # 1. Check word is not in stopwords
    if word not in stops:
      # 2. Check word part of speech (POS) and remove words.
      if upos not in ['PUNCT', 'ADP', 'NUM', 'VERB', 'ADJ'] and word not in toExclude or word in toInclude:
        lemma.append(word)
  return lemma


def lemmaDataSet(descriptions, toExclude, toInclude):
  lemmaDescriptions = []
  # Loop to lemmatize dish descriptions in DataSet
  for description in tqdm(descriptions):
    temp = lemmatizeSentence(description, toExclude, toInclude)
    lemmaDescriptions.append(temp)
  return lemmaDescriptions

In [ ]:
toExclude = ['grillir',
             'casa',
             'gramo',
             'base',
             'centro',
             'toque',
             'mix',
             'salsa',
             'negro']

toInclude = ['enyucado',
           'helado',
           'criollo',
           'parmesano',
           'costeño',
           'pomodoro',
           'prosciutto',
           'caramelizado',
           'champiñon',
           'artesanal',
           'boloñés',
           'balsámico',
           'césar',
           'salteado',
           'vegetal',
           'apanado',
           'asado',
           'cocido',
           'americano',
           'doble',
           'freído',
           'francés',
           'alemán',
           'crujiente',
           'italiano',
           'chantilly',
           'aguacate',
           'salami',
           'picante',
           'salado',
           'orégano',
           'cebollín',
           'bolognese',
           'mozzarella',
           'napolitano',
           'serrano',
           'cremoso',
           'ajinomoto']

## Main

In [ ]:
nlp = stanza.Pipeline(lang='es', processors='tokenize,mwt,pos,lemma', verbose = False)

In [ ]:
## CALL MAIN FUNCTION HERE
name = 'Ricardo Urrego'
restaurant = 'La Diva Trattoria'

# User most common words
most_common_words = func_user_likes(name, user_likes_df = user_likes_df, max_most_common=20)

# User matches with restaurant
user_dict = dict(most_common_words)
match_values = get_user_matches(restaurant, descriptions_df, user_dict)

# Print order in restaurant dishes
restaurant_dishes = descriptions_df.loc[descriptions_df['rest_name'] == restaurant]
restaurant_dishes = restaurant_dishes.reset_index(drop=True)
restaurant_dishes['user_likes'] = match_values
restaurant_dishes = restaurant_dishes.sort_values(by='user_likes', ascending=False)
user_descriptions = np.array(restaurant_dishes['dish_description'])[0:5]
#print(user_descriptions)

In [ ]:
columns = [name+' likes']
user_df_likes = pd.DataFrame(user_descriptions, columns=columns)
user_df_likes

,Ricardo Urrego likes
0,"Mozzarella de búfala y queso parmesano, salami..."
1,"Camarones, champiñon portobello, aceite de tru..."
2,"Lechuga verde rizada, rúgula, prosciutto, mozz..."
3,"Salsa pomodoro de la casa, queso mozzarella, a..."
4,"Salsa pomodoro de la casa, queso mozzarella, a..."


In [ ]:
file_name = name + '_' + restaurant + '.xlsx'
restaurant_dishes.to_excel(file_name)